In [64]:
import pandas as pd
import tensorflow as tf
import sklearn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import glob
#from glob import glob
from sklearn.utils import shuffle
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.metrics import Precision, Recall, AUC

Preprocess Data

In [65]:
#get the shape of one of the images
image_path = r"C:\Users\user\Anaconda\Programs\image_data\train\normal\KEMP_IMG_DATA_1.png"
img = Image.open(image_path)
img_width, img_height = img.size
img_channels = 3 if img.mode == 'RGB' else 1
print("Image Dimensions (Width, Height, Channels):", img_width, img_height, img_channels)

Image Dimensions (Width, Height, Channels): 480 720 1


Image Dataset Creation

In [67]:
class ImageClassification:
    def __init__(self,path1): #path1 represents the directory to the image dataset
        #creating dataframes (df) for the train and test sets
        self.train_df = self._create_df(path1, "train")
        self.test_df = self._create_df(path1, "test")

    #independent utilitity method
    @staticmethod
    #create dataframe with image path and the outcome (either normal or error)
    #construct the dataframes based on the paths to normal and error images found in the directory in path2
    def _create_df(path1, path2):
        #glob.glob is usedd to find all PNG files in the directories
        #files1 = glob.glob(os.path.join(path1+path2, "normal/*.png"))
        #files2 = glob.glob(os.path.join(path1+ path2, "error/*.png"))
        normal_path = os.path.join(path1, path2, "normal", "*png")
        error_path = os.path.join(path1, path2, "error", "*png")
        files1 = glob.glob(normal_path)
        files2 = glob.glob(error_path)
        print("Normal Path:", normal_path)
        print("Error Path:", error_path)
        #prints the number of normal and error images found
        print("normal: ", len(files1))
        print("error: ", len(files2))
            
        #constructs two data frames, df_n = error, df_p = normal
        df_n = pd.DataFrame()
        df_p = pd.DataFrame()
        df_n["name"] = [x for x in files2]
        df_n["outcome"] = 0.0
        df_p["name"] = [x for x in files1]
        df_p["outcome"] = 1.0
        #concatenates and shuffles the dataframes into a single dataframe 'df'
        df = pd.concat([df_n, df_p], axis=0, ignore_index=True)
        df = shuffle(df)
        return df

    #seperate X and Y to train your model
    #operates an instance of class self
    def create_x_and_y(self):
        #calls preprocessing method for each image path in train_df
        #creates an array 'X' of preprocessed images
        X = np.array([self.img_preprocess(p) for p in self.train_df.name.values])
        #retrieve the outcome column values from train_df
        Y = self.train_df.outcome.values

        print("Shape of X:", X.shape)
        return X, Y

    def test_x_and_y(self):
        #get the images in the test data
        X1 = np.array([self.img_preprocess(p) for p in self.test_df.name.values])
        #get the labels 0 and 1
        Y1 = self.test_df.outcome.values

        return X1, Y1

    @staticmethod
    def img_preprocess(image_path, target_size=(256, 256)):
        #load image
        img = Image.open(image_path)
        #convert to greyscale
        img = img.convert('L')
        #resize image
        img = img.resize(target_size)
        #convert to numpy array and reshape to 256, 256, 1
        img_array = np.array(img).reshape(target_size[0], target_size[1], 1)
        #convert to float32 and normalise value to be between 0 and 1
        img_array = img_array.astype('float32') / 255.0
        return img_array

#get current path of file
dir_path = os.path.join(os.getcwd(), "image_data")
#dir_path = r"C:\Users\user\Anaconda\Programs\Swinburne\archive\image_data"
print("Director Path :", dir_path)
aic = ImageClassification(path1=dir_path)

X, Y = aic.create_x_and_y()
X1, Y1 = aic.test_x_and_y()

Director Path : C:\Users\user\Anaconda\Programs\image_data
Normal Path: C:\Users\user\Anaconda\Programs\image_data\train\normal\*png
Error Path: C:\Users\user\Anaconda\Programs\image_data\train\error\*png
normal:  1102
error:  59
Normal Path: C:\Users\user\Anaconda\Programs\image_data\test\normal\*png
Error Path: C:\Users\user\Anaconda\Programs\image_data\test\error\*png
normal:  276
error:  15
Shape of X: (1161, 256, 256, 1)


Model Creation

In [68]:
#make variables
epochs = 20 #number of times training is done
input_shape = (256, 256, 1)
batch_size = 64

In [69]:
pipe_model = Sequential()
#first create convulutional layer
pipe_model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(256,256,1), padding='same'))
#than add another pooling layer
pipe_model.add(MaxPooling2D((2,2), padding='same'))
#add 2nd convulutional layer
pipe_model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
pipe_model.add(MaxPooling2D((2,2), padding='same'))
#flatten the vector before the fully connected layer
pipe_model.add(Flatten())
#dense layer (fcl)
pipe_model.add(Dense(1, activation='sigmoid'))

#compile the model, use binary cross entropy for the loss function and the adam optimizer
pipe_model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy', Precision(), Recall(), AUC()])

In [70]:
pipe_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 256, 256, 32)      320       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 128, 128, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 128, 128, 64)      18496     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 64, 64, 64)       0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 262144)            0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [71]:
X_train, Y_train = aic.create_x_and_y()

history = pipe_model.fit(X_train, Y_train,
                         epochs=10,  # Number of training epochs
                         batch_size=32,  # Number of samples per gradient update
                         validation_split=0.2)  # Fraction of training data to be used as validation data

Shape of X: (1161, 256, 256, 1)
Epoch 1/10
29/29 [==============================] - 20s 660ms/step - loss: 0.2009 - accuracy: 0.9213 - precision_5: 0.9499 - recall_5: 0.9682 - auc_5: 0.6944 - val_loss: 0.0966 - val_accuracy: 0.9614 - val_precision_5: 0.9609 - val_recall_5: 1.0000 - val_auc_5: 0.9966
Epoch 2/10
29/29 [==============================] - 18s 627ms/step - loss: 0.0887 - accuracy: 0.9741 - precision_5: 0.9745 - recall_5: 0.9989 - auc_5: 0.9361 - val_loss: 0.0444 - val_accuracy: 0.9957 - val_precision_5: 0.9955 - val_recall_5: 1.0000 - val_auc_5: 1.0000
Epoch 3/10
29/29 [==============================] - 18s 618ms/step - loss: 0.0254 - accuracy: 0.9935 - precision_5: 0.9932 - recall_5: 1.0000 - auc_5: 0.9974 - val_loss: 0.0127 - val_accuracy: 0.9957 - val_precision_5: 0.9955 - val_recall_5: 1.0000 - val_auc_5: 1.0000
Epoch 4/10
29/29 [==============================] - 18s 617ms/step - loss: 0.0142 - accuracy: 0.9968 - precision_5: 0.9966 - recall_5: 1.0000 - auc_5: 1.0000 - v

In [73]:
test_X, test_Y = aic.test_x_and_y()

test_eval = pipe_model.evaluate(test_X, test_Y, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])
print('Test precision:', test_eval[2])
print('Test recall:', test_eval[3])
print('Test AUC:', test_eval[4])

Test loss: 0.0026408066041767597
Test accuracy: 1.0
Test precision: 1.0
Test recall: 1.0
Test AUC: 1.0
